# Import Library

In [ ]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns

# BOBOT DENGAN AHP 

## Normalisasi

In [ ]:
def normalisasi_kriteria(kriteria):
  kriteria['Waktu Komputasi'] = kriteria['Waktu Komputasi']/sum(kriteria['Waktu Komputasi'])
  kriteria['Kompleksitas'] = kriteria['Kompleksitas']/sum(kriteria['Kompleksitas'])
  kriteria['Akurasi'] = kriteria['Akurasi']/sum(kriteria['Akurasi'])
  kriteria['Implementasi'] = kriteria['Implementasi']/sum(kriteria['Implementasi'])
  kriteria['Memori'] = kriteria['Memori']/sum(kriteria['Memori'])
  return kriteria

## Bobot Kriteria

In [ ]:
def bobot_kriteria(kriteria):
  # untuk kriteria yang sudah dinormalisasikan
  kriteria['Bobot'] = kriteria['Waktu Komputasi'] + kriteria['Kompleksitas'] + kriteria['Akurasi'] + kriteria['Implementasi'] + kriteria['Memori']
  kriteria['Bobot'] = kriteria['Bobot']/5
  return kriteria

## Mengecek Konsistensi

In [ ]:
def cek_konsistensi(kriteria, matrix_criteria,n):
  Ax = matrix_criteria.dot(kriteria['Bobot']) # A*X ; x = kriteria['Bobot']
  lamda_max = Ax.divide(kriteria['Bobot']) # Ax dibagi x, untuk mencari lamda max, x = kriteria['Bobot']

  lamda_max = lamda_max/n
  lamda_max = lamda_max.sum()
  
  ci = (lamda_max - n)/(n-1)# mencari CI
  ri = 1.12
  cr = ci/ri # mencari CR
  return "Consistent!" if cr<0.1 else "Not Consistent!"

## Fungsi AHP

In [ ]:
def ahp(matrix, n):
  kriteria = copy.deepcopy(matrix) # menyimpan nilai kriteria yang belum dinormalisasi
  kriteria = normalisasi_kriteria(kriteria) # normalisasi matrix kriteria
  kriteria = bobot_kriteria(kriteria) # menambah bobot pada kriteria
  print(cek_konsistensi(kriteria, matrix,n))
  return kriteria

# SAW

## Normalisasi matriks

In [ ]:
def normalisasi_saw(M_saw, kriteria):
  M_saw_normalized=np.zeros_like(M_saw, dtype=np.float64)
  for i in range(len(M_saw[0])):
      if (i+1) in kriteria['benefit']:
        M_saw_normalized[:,i] = M_saw[:,i]/M_saw[:,i].max(axis=0)
      else:
        M_saw_normalized[:,i] = M_saw[:,i].min(axis=0)/M_saw[:,i]
  return M_saw_normalized

## Penilaian dan Perangkingan

In [ ]:
def penilaian_saw(M_saw_normalized, w):
  # matriks dikali bobot
  M_saw_multiply = [np.multiply(i, w) for i in M_saw_normalized]
  M_saw_multiply = [sum(i) for i in M_saw_multiply]
  key = ['A'+str(i+1) for i in range(len(M_saw_multiply))]
  V = dict(zip(key,M_saw_multiply))
  V_sorted = dict(sorted(V.items(), key=lambda x:x[1], reverse=True))
  return V_sorted

## SAW main code

In [ ]:
def saw(matrix, w, kriteria):
  normalisasi_matrix = normalisasi_saw(matrix, kriteria) # normalisasi
  return penilaian_saw(normalisasi_matrix, w) # penilaian dan perangkingan

# TOPSIS

In [ ]:
def matrix_R(M):
  denom=[np.sqrt(np.sum(list(map(lambda x:x**2, M[:,i])))) for i in range(len(M[0]))]
  return M/denom

In [ ]:
def matrix_y(R,w):
  return R*w

In [ ]:
def nilai_A(y, kriteria):
  A_plus=[]; A_min=[]
  for i in range(len(y[0])):
    if i in kriteria['benefit']:
      A_plus.append(np.max(y[:,i]))
      A_min.append(np.min(y[:,i]))
    else:
      A_plus.append(np.min(y[:,i]))
      A_min.append(np.max(y[:,i]))
  
  return A_plus, A_min

In [ ]:
def nilai_D(A_plus, A_min, y):
  D_plus=np.array([np.sqrt(np.sum((A_plus-i)**2)) for i in y])
  D_min=np.array([np.sqrt(np.sum((i-A_min)**2)) for i in y])

  return D_plus, D_min

In [ ]:
def nilai_V(D_plus, D_min):
  value=D_min/(D_plus+D_min)
  key=['A'+str(i+1) for i in range(len(value))]
  V=dict(zip(key,value))
  V_sorted=dict(sorted(V.items(), key=lambda x:x[1], reverse=True))
  return V_sorted

In [ ]:
def TOPSIS(M,w,kriteria):
  R=matrix_R(M)
  y=matrix_y(R,w)
  A_plus, A_min=nilai_A(y, kriteria)
  D_plus, D_min=nilai_D(A_plus, A_min, y)
  V=nilai_V(D_plus, D_min)

  return V

In [ ]:
# print(TOPSIS(M,w,kriteria_topsis))

# COPELAND

In [ ]:
# data={
#     'DM1':{
#         'P1':0.911489,
#         'P9':0.571017,
#         'P10':0.553032,
#         'P8':0.530622,
#         'P4':0.521865,
#         'P7':0.521865,
#         'P5':0.462834,
#         'P3':0.45363,
#         'P2':0.435366,
#         'P6':0
#     },
#     'DM2':{
#         'P3':0.614761,
#         'P10':0.614761,
#         'P4':0.597624,
#         'P7':0.597624,
#         'P2':0.573689,
#         'P6':0.569533,
#         'P9':0.569533,
#         'P8':0.553453,
#         'P5':0.541074,
#         'P1':0.453388
#     },
#     'DM3':{
#         'P1':0.789999,
#         'P3':0.70706,
#         'P4':0.70706,
#         'P10':0.70706,
#         'P6':0.675952,
#         'P9':0.675952,
#         'P2':0.594851,
#         'P7':0.594851,
#         'P8':0.585358,
#         'P5':0.572409
#     },
#     'DM4':{
#         'P1':0.777171,
#         'P3':0.53296,
#         'P5':0.53296,
#         'P10':0.53296,
#         'P6':0.524957,
#         'P7':0.524957,
#         'P9':0.524957,
#         'P2':0.522668,
#         'P4':0.522668,
#         'P8':0.522668
#     },
# }

In [ ]:
def ranking_DM(data):
  ranking={i:dict(zip(data[i].keys(), np.arange(1,len(data[i])+1))) for i in data}
  return ranking

def pairwise_contest(DM_weight, ranking):
  voting={'A'+str(i):{'win':0, 'draw':0, 'lose':0} for i in range(1,len(data['DM1'])+1)};
  for i in range(1, len(ranking['DM1'])+1):
    for j in range(i+1, len(ranking['DM1'])+1):
      sum=[0,0]
      for k in ranking:
        if ranking[k]['A'+str(i)]<ranking[k]['A'+str(j)]:
          sum[0]+=DM_weight[k]
        else:
          sum[1]+=DM_weight[k]
      
      if sum[0]>sum[1]:
        voting['A'+str(i)]['win']+=1
        voting['A'+str(j)]['lose']+=1
      elif sum[0]<sum[1]:
        voting['A'+str(j)]['win']+=1
        voting['A'+str(i)]['lose']+=1
      else:
        voting['A'+str(j)]['draw']+=1
        voting['A'+str(i)]['draw']+=1
    
  return voting

def decision(voting):
  poin={i:voting[i]['win']-voting[i]['lose'] for i in voting}
  final_rank=dict(sorted(poin.items(), key=lambda x:x[1], reverse=True))
  return list(final_rank.keys())

In [ ]:
def pairwise_contest(DM_weight, ranking):
  voting={'A'+str(i):{'win':0, 'draw':0, 'lose':0} for i in range(1,len(data['DM1'])+1)};
  for i in range(1, len(ranking['DM1'])+1):
    for j in range(i+1, len(ranking['DM1'])+1):
      sum=[0,0]
      for k in ranking:
        if ranking[k]['A'+str(i)]<ranking[k]['A'+str(j)]:
          sum[0]+=DM_weight[k]
        else:
          sum[1]+=DM_weight[k]
      
      if sum[0]>sum[1]:
        voting['A'+str(i)]['win']+=1
        voting['A'+str(j)]['lose']+=1
      elif sum[0]<sum[1]:
        voting['A'+str(j)]['win']+=1
        voting['A'+str(i)]['lose']+=1
      else:
        voting['A'+str(j)]['draw']+=1
        voting['A'+str(i)]['draw']+=1
  
  return voting

In [ ]:
def decision(voting):
  poin={i:voting[i]['win']-voting[i]['lose'] for i in voting}
  final_rank=dict(sorted(poin.items(), key=lambda x:x[1], reverse=True))
  return list(final_rank.keys())


In [ ]:
def copeland(data, DM_weight):
  ranking=ranking_DM(data)
  voting=pairwise_contest(DM_weight, ranking)

  for k,v in voting.items():
    print(k,":",v)
  return decision(voting)

In [ ]:
# DM_weight={'DM1':.1, 'DM2':.4, 'DM3':.3, 'DM4':.2}
# copeland(data, DM_weight)

# INTEGRASI

## DM_Iris

In [ ]:
# matriks kriteria
kriteria=pd.DataFrame({
    'Waktu Komputasi':[1, 5, 0.2, 0.3, 0.3], 
    'Kompleksitas':[0.2, 1, 0.2, 0.3, 0.3],
    "Akurasi": [5, 5, 1, 3, 1], 
    "Implementasi": [3, 3, 0.3, 1, 0.3],
    "Memori" : [3, 3, 1, 3, 1]
}, index = ["Waktu Komputasi","Kompleksitas","Akurasi", "Implementasi", "Memori"])

# menyimpan nilai kriteria yang belum dinormalisasi
kriteria=ahp(kriteria, 5)
kriteria

Consistent!


,Waktu Komputasi,Kompleksitas,Akurasi,Implementasi,Memori,Bobot
Waktu Komputasi,0.147059,0.10,0.333333,0.394737,0.272727,0.249571
Kompleksitas,0.735294,0.50,0.333333,0.394737,0.272727,0.447218
Akurasi,0.029412,0.10,0.066667,0.039474,0.090909,0.065292
Implementasi,0.044118,0.15,0.200000,0.131579,0.272727,0.159685
Memori,0.044118,0.15,0.066667,0.039474,0.090909,0.078233


In [ ]:
# nilai kriteria dari dara IRIS
# urutan :::
# Logistic Regression
# Decision Tree
# Naive Bayes
# SVM
# KNN
# MLP
# Random Forest
# XGboost

M_saw=[
       [1, 165130240, 97, 3, 0.1],
       [1, 165048320, 93, 3, 0.2],
       [1, 165212160, 85, 3, 0.2],
       [1, 165908480, 98, 1, 0.2],
       [1.2, 165908480, 97, 4, 0.3],
       [1.5, 220208480, 97.5, 1, 0.5],
       [1, 172208480, 93, 2, 0.24],
       [1.5, 147108180, 95, 2, 0.22]
]
w=[round(val, 2) for val in kriteria['Bobot'].tolist()]
kriteria_saw={
  'benefit':[3,4],
  'cost':[1,2,5]
}
M_saw =np.array(M_saw)
DM_Iris=saw(M_saw, w, kriteria_saw)

{'A1': 0.9201734075392335, 'A2': 0.877515241077799, 'A3': 0.8714031998959053, 'A5': 0.8632929319999625, 'A7': 0.8141720162732542, 'A8': 0.8008874458874459, 'A4': 0.7990072177142484, 'A6': 0.5929277425084586}


In [ ]:
for k,v in DM_Iris.items():
  print(k,":",v)

A1 : 0.9201734075392335
A2 : 0.877515241077799
A3 : 0.8714031998959053
A5 : 0.8632929319999625
A7 : 0.8141720162732542
A8 : 0.8008874458874459
A4 : 0.7990072177142484
A6 : 0.5929277425084586


## DM_Wine

In [ ]:
Criteria=pd.DataFrame({
    'Waktu Komputasi':[1., .5, 1., .25, .33],
    'Kompleksitas':[2., 1., 3., .25, .5],
    'Akurasi':[1.,.3,1.,.2,.25],
    'Implementasi':[4., 4., 5., 1., 2.],
    'Memori':[3., 2., 4., .5, 1.]},
    index=["Waktu Komputasi","Kompleksitas","Akurasi", "Implementasi", "Memori"])

kriteria = ahp(Criteria, 5)
kriteria

Consistent!


,Waktu Komputasi,Kompleksitas,Akurasi,Implementasi,Memori,Bobot
Waktu Komputasi,0.324675,0.296296,0.363636,0.2500,0.285714,0.304064
Kompleksitas,0.162338,0.148148,0.109091,0.2500,0.190476,0.172011
Akurasi,0.324675,0.444444,0.363636,0.3125,0.380952,0.365242
Implementasi,0.081169,0.037037,0.072727,0.0625,0.047619,0.060210
Memori,0.107143,0.074074,0.090909,0.1250,0.095238,0.098473


In [ ]:
Matrix=pd.DataFrame({
    'Waktu Komputasi':[134.0, 27.8, 7.57, 191., 24.8, 278, 2380., 825.],
    'Kompleksitas':[13189.0, 40606.55, 13189.0, 15813611.0, 39567.0, 4060654.62, 116363698176.0, 3691.5],
    'Akurasi':[0.515, 1.0, 0.4625, 0.5075, 0.7875, 0.5575, 0.6125, 0.7925],
    'Implementasi':[4, 3, 4, 3, 2, 2, 1, 3],
    'Memori':[1, 81, 2, 190, 314, 197, 36, 360]},
    index=['LogReg', 'Decision Tree', 'Naive Bayes', 'SVM', 'KNN', 'Random Forest', 'MLP', 'XGBoost'])

M=np.array(Matrix.values.tolist()) # list kriteria dan nilainya
w=[round(val, 2) for val in kriteria['Bobot'].tolist()] # input bobot dari perhitungan ahp
kriteria_topsis={
  'benefit':[3,4],
  'cost':[1,2,5]
}

DM_Wine=TOPSIS(M,w,kriteria_topsis)
# print(DM_Wine)

{'A4': 0.8928334537967376, 'A6': 0.8647470043301816, 'A3': 0.8427321332095179, 'A5': 0.8366905186035767, 'A1': 0.8311092754419239, 'A2': 0.7383376073351332, 'A8': 0.6932938523370872, 'A7': 0.1834097931393461}


In [ ]:
for k,v in DM_Wine.items():
  print(k,":",v)

A4 : 0.8928334537967376
A6 : 0.8647470043301816
A3 : 0.8427321332095179
A5 : 0.8366905186035767
A1 : 0.8311092754419239
A2 : 0.7383376073351332
A8 : 0.6932938523370872
A7 : 0.1834097931393461


## GDSS

In [ ]:
data={'DM1':DM_Iris, 'DM2':DM_Wine}
DM_weight={'DM1':.45, 'DM2':.55}
Decision=copeland(data, DM_weight)

{'A1': {'win': 3, 'draw': 0, 'lose': 4}, 'A2': {'win': 2, 'draw': 0, 'lose': 5}, 'A3': {'win': 5, 'draw': 0, 'lose': 2}, 'A4': {'win': 7, 'draw': 0, 'lose': 0}, 'A5': {'win': 4, 'draw': 0, 'lose': 3}, 'A6': {'win': 6, 'draw': 0, 'lose': 1}, 'A7': {'win': 0, 'draw': 0, 'lose': 7}, 'A8': {'win': 1, 'draw': 0, 'lose': 6}}
A1 : {'win': 3, 'draw': 0, 'lose': 4}
A2 : {'win': 2, 'draw': 0, 'lose': 5}
A3 : {'win': 5, 'draw': 0, 'lose': 2}
A4 : {'win': 7, 'draw': 0, 'lose': 0}
A5 : {'win': 4, 'draw': 0, 'lose': 3}
A6 : {'win': 6, 'draw': 0, 'lose': 1}
A7 : {'win': 0, 'draw': 0, 'lose': 7}
A8 : {'win': 1, 'draw': 0, 'lose': 6}


In [ ]:
Model={
    'A1':'Logistic Regression',
    'A2':'Decision Tree',
    'A3':'Naive Bayes',
    'A4':'SVM',
    'A5':'KNN',
    'A6':'MLP',
    'A7':'Random Forest',
    'A8':'XGBoost'
}
print('Urutan Rekomendasi berdasarkan GDSS: \n', [Model[i] for i in Decision])

Urutan Rekomendasi berdasarkan GDSS: 
 ['SVM', 'MLP', 'Naive Bayes', 'KNN', 'Logistic Regression', 'Decision Tree', 'XGBoost', 'Random Forest']
